<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Predicción de Aterrizaje de la Primera Etapa del Falcon 9 de SpaceX


## Laboratorio 2: Procesamiento de Datos


Tiempo estimado necesario: **60** minutos


En este laboratorio, realizaremos un Análisis Exploratorio de Datos (EDA) para identificar patrones en los datos y determinar cuál será la etiqueta para entrenar modelos supervisados.

En el conjunto de datos, hay varios casos en los que el propulsor no aterrizó con éxito. En ocasiones, se intentó el aterrizaje, pero fracasó debido a un accidente. Por ejemplo, <code>True Ocean</code> significa que el resultado de la misión fue un aterrizaje exitoso en una región específica del océano, mientras que <code>False Ocean</code> significa que el resultado de la misión fue un aterrizaje fallido en una región específica del océano. <code>True RTLS</code> indica un aterrizaje exitoso en una plataforma terrestre, y <code>False RTLS</code> significa que el aterrizaje en una plataforma terrestre fue infructuoso. <code>True ASDS</code> denota un aterrizaje exitoso en una nave no tripulada, mientras que <code>False ASDS</code> señala un aterrizaje no exitoso en una nave no tripulada.

En este laboratorio, principalmente convertiremos esos resultados en etiquetas de entrenamiento, donde `1` significa que el propulsor aterrizó con éxito y `0` significa que no lo hizo.


La primera etapa del Falcon 9 aterrizará con éxito.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Se muestran varios ejemplos de aterrizajes fallidos aquí:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objetivos
Realizar un Análisis Exploratorio de Datos y determinar Etiquetas de Entrenamiento

- Análisis Exploratorio de Datos
- Determinar Etiquetas de Entrenamiento


----


## Importar bibliotecas y definir funciones auxiliares


Importaremos las siguientes bibliotecas.

In [2]:
# Pandas es una biblioteca de software escrita para el lenguaje de programación Python para la manipulación y análisis de datos.
import pandas as pd
# NumPy es una biblioteca para el lenguaje de programación Python que añade soporte para matrices y matrices multidimensionales, junto con una amplia colección de funciones matemáticas de alto nivel para operar en estas matrices.
import numpy as np


### Análisis de datos


Carga el conjunto de datos de SpaceX de la última sección.


In [3]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identifica y calcula el porcentaje de valores faltantes en cada atributo.


In [4]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identifica qué columnas son numéricas y cuáles son categóricas:


In [5]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### TAREA 1: Calcular el número de lanzamientos en cada sitio

Los datos contienen varias instalaciones de lanzamiento de SpaceX: el Complejo de Lanzamiento Espacial 40 de Cape Canaveral, VAFB SLC 4E en la Base de la Fuerza Aérea de Vandenberg, el Complejo de Lanzamiento 39A del Centro Espacial Kennedy. La ubicación de cada lanzamiento está en la columna <code>LaunchSite</code>.

A continuación, veamos el número de lanzamientos para cada sitio.

Utiliza el método <code>value_counts()</code> en la columna <code>LaunchSite</code> para determinar el número de lanzamientos en cada sitio:


In [6]:
# Aplica value_counts() en la columna LaunchSite

df['LaunchSite'].value_counts()


LaunchSite
CCAFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: count, dtype: int64

Cada lanzamiento apunta a una órbita específica, y aquí hay algunos tipos comunes de órbitas:


* <b>LEO</b>: Órbita baja terrestre (LEO) es una órbita centrada en la Tierra con una altitud de 2,000 km (1,200 millas) o menos (aproximadamente un tercio del radio de la Tierra), o con al menos 11.25 períodos por día (un período orbital de 128 minutos o menos) y una excentricidad menor de 0.25. La mayoría de los objetos fabricados por el hombre en el espacio exterior están en LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Las órbitas muy bajas de la Tierra (VLEO) pueden definirse como órbitas con una altitud media por debajo de 450 km. Operar en estas órbitas puede proporcionar varios beneficios a las naves espaciales de observación terrestre, ya que la nave opera más cerca de la observación <a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.

* <b>GTO</b>: Una órbita geoestacionaria es una órbita terrestre alta que permite a los satélites igualar la rotación de la Tierra. Ubicada a 35,786 kilómetros (22,236 kilómetros) sobre el ecuador de la Tierra, esta posición es un lugar valioso para monitorear el clima, las comunicaciones y la vigilancia. Debido a que el satélite orbita a la misma velocidad que la Tierra está girando, el satélite parece quedarse en su lugar sobre un solo meridiano, aunque puede moverse de norte a sur," escribió la NASA en su sitio web Earth Observatory <a  href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.

* <b>SSO (o SO)</b>: Es una órbita síncrona con el sol, también llamada órbita heliosíncrona, es una órbita casi polar alrededor de un planeta, en la que el satélite pasa sobre cualquier punto dado de la superficie del planeta a la misma hora solar media local <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.

* <b>ES-L1</b>: En los puntos de Lagrange, las fuerzas gravitacionales de los dos cuerpos grandes se anulan de tal manera que un pequeño objeto colocado en órbita allí está en equilibrio con respecto al centro de masa de los cuerpos grandes. L1 es uno de estos puntos entre el sol y la Tierra <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a>.

* <b>HEO</b>: Una órbita altamente elíptica, es una órbita elíptica con alta excentricidad, generalmente en referencia a la Tierra <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.

* <b>ISS</b>: Una estación espacial modular (satélite artificial habitable) en órbita baja terrestre. Es un proyecto de colaboración multinacional entre cinco agencias espaciales participantes: NASA (Estados Unidos), Roscosmos (Rusia), JAXA (Japón), ESA (Europa) y CSA (Canadá) <a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>.

* <b>MEO</b>: Órbitas geocéntricas que van desde 2,000 km (1,200 millas) hasta justo debajo de la órbita geoestacionaria a 35,786 kilómetros (22,236 millas). También conocida como órbita circular intermedia. Estas órbitas son "más comúnmente a 20,200 kilómetros (12,600 millas) o 20,650 kilómetros (12,830 millas), con un período orbital de 12 horas <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>.

* <b>HEO</b>: Órbitas geocéntricas por encima de la altitud de la órbita geoestacionaria (35,786 km o 22,236 millas) <

a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>.

* <b>GEO</b>: Es una órbita geoestacionaria circular a 35,786 kilómetros (22,236 millas) sobre el ecuador de la Tierra y siguiendo la dirección de la rotación de la Tierra <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>.

* <b>PO</b>: Es uno de los tipos de satélites en los que un satélite pasa por encima o casi por encima de ambos polos del cuerpo orbitalizado (generalmente un planeta como la Tierra <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>.

Algunos se muestran en el siguiente gráfico:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### TAREA 2: Calcula el número y la ocurrencia de cada órbita


Utiliza el método <code>.value_counts()</code> para determinar el número y la ocurrencia de cada órbita en la columna <code>Orbit</code>.


In [7]:
# Aplica value_counts en la columna Orbit
df['Orbit'].value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: count, dtype: int64

### TAREA 3: Calcular el número y la frecuencia del resultado de la misión de las órbitas


Usa el método <code>.value_counts()</code> en la columna <code>Outcome</code> para determinar el número de <code>landing_outcomes</code>. Luego, asígnalo a una variable llamada <code>landing_outcomes</code>.

In [8]:
# landing_outcomes = values on Outcome column
landing_outcomes = df['Outcome'].value_counts()


<code>True Ocean</code> significa que el resultado de la misión aterrizó con éxito en una región específica del océano, mientras que <code>False Ocean</code> significa que el resultado de la misión no aterrizó con éxito en una región específica del océano. <code>True RTLS</code> significa que el resultado de la misión aterrizó con éxito en una plataforma terrestre, <code>False RTLS</code> significa que el resultado de la misión no aterrizó con éxito en una plataforma terrestre. <code>True ASDS</code> significa que el resultado de la misión aterrizó con éxito en un barco no tripulado, <code>False ASDS</code> significa que el resultado de la misión no aterrizó con éxito en un barco no tripulado. <code>None ASDS</code> y <code>None None</code> representan un fallo en el aterrizaje.


In [9]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


Creamos un conjunto de resultados donde la segunda etapa no aterrizó exitosamente:


In [10]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### TAREA 4: Crea una etiqueta de resultado de aterrizaje a partir de la columna de resultado


Usando la columna <code>Outcome</code>, crea una lista donde el elemento sea cero si la fila correspondiente en <code>Outcome</code> está en el conjunto <code>bad_outcome</code>; de lo contrario, será uno. Luego, asígnalo a la variable <code>landing_class</code>:


In [11]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
landing_class = [0 if outcome in bad_outcomes else 1 for outcome in df['Outcome']]


Esta variable representará la variable de clasificación que indica el resultado de cada lanzamiento. Si el valor es cero, significa que la primera etapa no aterrizó con éxito; uno significa que la primera etapa aterrizó con éxito.


In [12]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [13]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Podemos utilizar la siguiente línea de código para determinar la tasa de éxito:

In [14]:
df["Class"].mean()

0.6666666666666666

Eso indica que aproximadamente el 66.67% de los lanzamientos tuvieron éxito en el aterrizaje de la primera etapa.

Ahora podemos exportarlo a un archivo CSV para la próxima sección, pero para mantener las respuestas consistentes, en el próximo laboratorio proporcionaremos datos en un rango de fechas preseleccionado.


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


In [15]:
df.to_csv("dataset_part_2.csv", index=False)

## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |


Copyright © 2021 IBM Corporation. All rights reserved.
